In [356]:
import pandas as pd
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LinearRegression
from sklearn_pandas import DataFrameMapper, cross_val_score
from sklearn_pandas.pipeline 

ImportError: cannot import name 'Pipeline' from 'sklearn_pandas' (/Users/krex/anaconda3/lib/python3.7/site-packages/sklearn_pandas/__init__.py)

In [24]:
df = pd.read_json('prepped_data.json').sort_index()

In [26]:
df.head()

,age,num_comments,score,text,timestamp,weekday_posted,hour_posted,log_score,log_comments
0,300,1588,21720,China Killing Prisoners To Harvest Organs For ...,2019-06-19 11:49:08,2,11,9.985989,7.370237
1,240,402,2661,Muslim family dragged out of Belgian embassy i...,2019-06-19 12:05:31,2,12,7.886457,5.996477
2,660,3320,46977,Women outperform men after Japan medical schoo...,2019-06-19 05:51:44,2,5,10.757413,8.107723
3,360,202,1474,MH17 crash: Investigators 'to charge four with...,2019-06-19 10:50:51,2,10,7.295735,5.308317
4,660,1336,2665,Iranian official calls on world to unite again...,2019-06-19 05:09:15,2,5,7.887959,7.197443


In [68]:
X = df[['text','age','weekday_posted','hour_posted']]
y = df['log_score']

In [279]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=2019)


In [280]:
X_train.head()

,text,age,weekday_posted,hour_posted
4867,Donald Trump Jr. on Sunday claimed CNN is cove...,1440,0,4
1395,"Oxfam warns of the ""worst cholera outbreak in ...",1440,5,11
2819,Exposure to weed killing products increases ri...,1440,3,11
567,Hong Kong protesters demand China be held to a...,660,0,0
1444,Austrian Government Seeks to Eliminate Interne...,180,4,10


## Confirm if we can tfidf corpus first, dummy 

In [242]:
count_vec = CountVectorizer()
X_train_counts = count_vec.fit_transform(X_train['text'])
X_train_counts.shape

(3631, 10230)

In [203]:
X_train.shape, X_train_counts.shape

((3631, 4), (3631, 10230))

### Make one Pipeline per feature type. the pipeline can't transform each on its own

In [231]:
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer

In [268]:
num_transformer = Pipeline(steps=[
    ('scaler',StandardScaler()),
])

cat_transformer = Pipeline(steps=[
    ('OneHot',OneHotEncoder(drop='first',categories='auto',handle_unknown='error'))
])
text_transformer = Pipeline(steps=[
    ('vect',CountVectorizer()),
    #('tfidf',TfidfTransformer())
])

In [233]:
# preprocessor = ColumnTransformer(
#     [('onehot_day', OneHotEncoder(drop='first', handle_unknown='error',categories='auto'), ['weekday_posted']),
#      ('ohe_hour', OneHotEncoder(drop='first', handle_unknown='error',categories='auto'), ['hour_posted']),
#      ('age_scaler',StandardScaler() , ['age']),
#      ('text_tfidf', TfidfVectorizer(), ['text'])],
#     remainder='passthrough')

In [283]:
X_train = X_train.reset_index(drop=True)

In [346]:
num_features = ['age']
cat_features = [['weekday_posted','hour_posted']]
text_features = ['text']

## Cristian: testing

In [350]:
# cn
preprocessor = ColumnTransformer(
    transformers = [
        ('num', num_transformer, num_features),
        ('wp', OneHotEncoder(drop='first', categories='auto'), ["weekday_posted"]),
        ('hp', OneHotEncoder(drop='first', categories='auto'), ["hour_posted"])
        #('text',text_transformer, text_features)
    ])

In [351]:
# cn
cn_pipe = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('union', ColumnTransformer(
        [
            #('vect',CountVectorizer(), 0),
            ('tfidf', TfidfVectorizer(), 0)
        ]
    )),
    ('regressor', LinearRegression())],
                  verbose=True)

In [354]:
# cn asdasd
cn_pipe.fit(X_train, y_train)

[Pipeline] ...... (step 1 of 3) Processing preprocessor, total=   0.0s


AttributeError: lower not found

In [344]:
# cn
cn_pipe.predict(X_train)

array([10.3709254 , 10.25944725, 10.67009429, ...,  9.66294348,
        8.20166021,  9.40021665])

In [345]:
cn_pipe.predict(X_train).shape

(3631,)

In [289]:
preprocessor = ColumnTransformer(
    transformers = [
        ('num', num_transformer, num_features),
        ('cat', cat_transformer, cat_features),
        ('text',text_transformer, text_features)
    ])

In [290]:
lr_pipe = Pipeline(steps=[('preprocessor', preprocessor),
                         ('regressor', LinearRegression())])

In [335]:
X_train.iloc[:5, 0]

0    Donald Trump Jr. on Sunday claimed CNN is cove...
1    Oxfam warns of the "worst cholera outbreak in ...
2    Exposure to weed killing products increases ri...
3    Hong Kong protesters demand China be held to a...
4    Austrian Government Seeks to Eliminate Interne...
Name: text, dtype: object

In [292]:
lr_pipe.fit(X_train, y_train)

ValueError: blocks[0,:] has incompatible row dimensions. Got blocks[0,2].shape[0] == 1, expected 3631.

In [266]:
lr_pipe.predict(X_test).shape

(1211,)

In [267]:
X_test.shape

(1211, 4)